In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
import keras
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.optimizers import adam_v2
from sklearn.model_selection import train_test_split
import keras_metrics
from keras import metrics
from keras import backend as K
import keras
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [2]:
df_train = pd.read_csv('train_preprocessed-nonPCA.csv',index_col=[0])
df_test = pd.read_csv('test_preprocessed-nonPCA.csv',index_col=[0])

In [18]:
X_train=df_train.iloc[:,:-1].values
y_train=df_train.iloc[:,-1].values
X_test=df_test.iloc[:,:-1].values
y_test=df_test.iloc[:,-1].values

# print the shape of the split dataset
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(40736, 118)
(40736,)
(10184, 118)
(10184,)


In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

In [21]:
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


class KerasCNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, build_fn=None, **kwargs):
        self.build_fn = build_fn
        self.kwargs = kwargs
        self.model = None

    def fit(self, X, y, **kwargs):
        if self.build_fn is None:
            raise ValueError("Cannot build Keras model: build_fn is None")

        # Create a KerasClassifier, which is a scikit-learn wrapper around Keras models
        self.model = KerasClassifier(build_fn=self.build_fn, **self.kwargs)
        
        # Fit the Keras model
        self.model.fit(X, y, **kwargs)
        
        # Set the classes_ attribute
        self.classes_ = np.unique(y)
        
        return self

    def predict(self, X, **kwargs):
        # Use the Keras model to make predictions
        return self.model.predict(X, **kwargs)

    def predict_proba(self, X, **kwargs):
        # Use the Keras model to make probability predictions
        return self.model.predict_proba(X, **kwargs)

    def score(self, X, y, **kwargs):
        # Use the score method of the Keras model
        return self.model.score(X, y, **kwargs)

# You must define your CNN architecture in a function
def create_cnn_model():
    # Define your CNN model architecture here
    # ...
    return model

In [22]:
estimators = [
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=6, random_state=42)),
    ('cnn', KerasCNNClassifier(build_fn=create_cnn_model, epochs=10, batch_size=32, verbose=0))  # Adjust these parameters as needed
]

# Create the stacking model
stacking_classifier = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(C=0.1)
)

# Train the stacking model
stacking_classifier.fit(X_train, y_train)

   6/1273 [..............................] - ETA: 14s - loss: 1.2464 - accuracy: 0.4635

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  app.launch_new_instance()


   7/1019 [..............................] - ETA: 8s - loss: 1.0830 - accuracy: 0.5268 

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  app.launch_new_instance()


   8/1019 [..............................] - ETA: 7s - loss: 1.2108 - accuracy: 0.4844

...

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  app.launch_new_instance()


   8/1019 [..............................] - ETA: 8s - loss: 1.2028 - accuracy: 0.4727

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  app.launch_new_instance()


255/255 [==============================] - 1s 4ms/step


StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(min_samples_split=6,
                                                       n_estimators=200,
                                                       random_state=42)),
                               ('cnn',
                                KerasCNNClassifier(build_fn=<function create_cnn_model at 0x0000017900807EE8>))],
                   final_estimator=LogisticRegression(C=0.1))

In [23]:
y_pred = stacking_classifier.predict(X_test)

319/319 [==============================] - 2s 5ms/step


In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
def evaluate(y_pred, y_test):
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
evaluate(y_pred, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.67      0.61      3400
           1       0.42      0.01      0.02      2028
           2       0.42      0.66      0.51      3162
           3       0.58      0.38      0.46      1594

    accuracy                           0.49     10184
   macro avg       0.49      0.43      0.40     10184
weighted avg       0.49      0.49      0.44     10184



In [25]:
from sklearn.preprocessing import label_binarize
y_bin = label_binarize(y_test, classes=[0, 1, 2, 3])
y_pred_proba = stacking_classifier.predict_proba(X_test)
from sklearn.metrics import roc_auc_score
auroc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr", average="macro")
print(f"Multi-class AUROC (One-vs-Rest): {auroc}")

319/319 [==============================] - 1s 3ms/step
Multi-class AUROC (One-vs-Rest): 0.7220348863483614


In [6]:
estimators = [
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=6,random_state=42)),
    ('gf', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42))
]

# 创建堆叠模型
stacking_classifier = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(C=0.1)
)

# 训练堆叠模型
stacking_classifier.fit(X_train, y_train)

D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(min_samples_split=6,
                                                       n_estimators=200,
                                                       random_state=42)),
                               ('gf',
                                GradientBoostingClassifier(learning_rate=1.0,
                                                           max_depth=1,
                                                           random_state=42))],
                   final_estimator=LogisticRegression(C=0.1))

In [7]:
y_pred = stacking_classifier.predict(X_test)

In [8]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
def evaluate(y_pred, y_test):
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
evaluate(y_pred, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.67      0.60      3400
           1       0.50      0.01      0.01      2028
           2       0.42      0.65      0.51      3162
           3       0.55      0.37      0.44      1594

    accuracy                           0.49     10184
   macro avg       0.50      0.42      0.39     10184
weighted avg       0.50      0.49      0.43     10184



In [9]:
from sklearn.preprocessing import label_binarize
y_bin = label_binarize(y_test, classes=[0, 1, 2, 3])
y_pred_proba = stacking_classifier.predict_proba(X_test)
from sklearn.metrics import roc_auc_score
auroc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr", average="macro")
print(f"Multi-class AUROC (One-vs-Rest): {auroc}")


Multi-class AUROC (One-vs-Rest): 0.7165031897058827


In [6]:
estimators = [
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=6,random_state=42)),
    ('dt', DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=5,min_samples_split=5))
]

# 创建堆叠模型
stacking_classifier = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(C=0.1)
)

# 训练堆叠模型
stacking_classifier.fit(X_train, y_train)

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(min_samples_split=6,
                                                       n_estimators=200,
                                                       random_state=42)),
                               ('dt',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=10,
                                                       min_samples_leaf=5,
                                                       min_samples_split=5))],
                   final_estimator=LogisticRegression(C=0.1))

In [8]:
y_pred = stacking_classifier.predict(X_test)

In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
def evaluate(y_pred, y_test):
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
evaluate(y_pred, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.67      0.60      3400
           1       0.38      0.00      0.00      2028
           2       0.41      0.64      0.50      3162
           3       0.54      0.36      0.43      1594

    accuracy                           0.48     10184
   macro avg       0.47      0.42      0.38     10184
weighted avg       0.47      0.48      0.42     10184



In [10]:
from sklearn.preprocessing import label_binarize
y_bin = label_binarize(y_test, classes=[0, 1, 2, 3])
y_pred_proba = stacking_classifier.predict_proba(X_test)
from sklearn.metrics import roc_auc_score
auroc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr", average="macro")
print(f"Multi-class AUROC (One-vs-Rest): {auroc}")

Multi-class AUROC (One-vs-Rest): 0.7079736201274476
